## Import

Lorem ipsum dolor sit amet.

In [1]:
import scipy
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

## Nahrání signálu

Lorem ipsum dolor sit amet.

In [35]:
# NEZAPOMENOUT ŘEŠIT OBA KANÁLY

class Instrument:
    def __init__(self, name, directory) -> None:
        self.name = name
        self.directory = directory
        self.wav_files = [file for file in os.listdir(directory) if file.endswith('.wav')]
        self.signals_raw = {}
        self.signals_envelope = {}
        self.signals_spectogram = {}
        self._load_signals()

    def _load_signals(self):
        for wav_file in self.wav_files:
            file_path = os.path.join(self.directory, wav_file)
            sample_rate, data = wavfile.read(file_path)
            self.signals_raw[wav_file] = (sample_rate, data) # Store the sample rate and signal data as a tuple (sample_rate, data)
    

all_instrument_classes = []

all_instrument_classes.append(Instrument("cello", "Dataset\\cel"))
all_instrument_classes.append(Instrument("clarinet", "Dataset\\cla"))
all_instrument_classes.append(Instrument("flute", "Dataset\\flu"))
all_instrument_classes.append(Instrument("guitar_acoustic", "Dataset\\gac"))
all_instrument_classes.append(Instrument("guitar_electric", "Dataset\\gel"))
all_instrument_classes.append(Instrument("organ", "Dataset\\org"))
all_instrument_classes.append(Instrument("piano", "Dataset\\pia"))
all_instrument_classes.append(Instrument("saxophone", "Dataset\\sax"))
all_instrument_classes.append(Instrument("trumpet", "Dataset\\tru"))
all_instrument_classes.append(Instrument("violin", "Dataset\\vio"))
all_instrument_classes.append(Instrument("voice", "Dataset\\voi"))



## Očištění a sjednocení signálu

Lorem ipsum dolor sit amet.

In [ ]:
# NEZAPOMENOUT ŘEŠIT OBA KANÁLY

# Sjednocení
## Všechna trénovací data mají stejný sample_rate(44100) i délku(3s) i data_shape(132299, 2), takže není třeba sjednocovat.

# Očištění
# Audio je kvalitní a žádné závažné problémy, které by bylo třeba odstranit

## Amplitude envelope

Lorem ipsum dolor sit amet.

In [37]:
# RMS (Root Mean Square) Envelope, because i found it is best for continuous sounds (instruments)
def amplitude_envelope(signal, sample_rate, window_size=0.05):
    window_size_in_samples = int(window_size * sample_rate)
    envelope = []
    for i in range(0, len(signal), window_size_in_samples):
        window = signal[i:i + window_size_in_samples]
        rms = np.sqrt(np.mean(window**2)) if len(window) > 0 else 0
        envelope.append(rms)
    return np.array(envelope)


for instrument in all_instrument_classes:
    for wav_file, (sample_rate, signal) in instrument.signals_raw.items():
        instrument.signals_envelope[wav_file] = amplitude_envelope(signal, sample_rate)

C:\Users\popa4\AppData\Local\Temp\ipykernel_14188\2639312768.py:7: RuntimeWarning: invalid value encountered in sqrt
  rms = np.sqrt(np.mean(window**2)) if len(window) > 0 else 0  # Avoid errors with empty windows


## Spectrogram

Lorem ipsum dolor sit amet.

In [43]:
# spectrogram se počítá skrz FFT (fft = frekvence místo amplitudy, neural network mňamuje)
'''
Přes librosu to nejde, divný dependencies na starší verze numpy

def mel_spectrogram(signal, sample_rate, n_mels=128, n_fft=2048, hop_length=512):
    mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sample_rate, n_mels=n_mels, 
                                                         n_fft=n_fft, hop_length=hop_length)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    return mel_spectrogram_db
'''

def hz_to_mel(hz):
    return 2595 * np.log10(1 + hz / 700)

def mel_to_hz(mel):
    return 700 * (10**(mel / 2595) - 1)

def mel_filter_bank(sample_rate, n_fft, n_mels):
    mel_min = 0
    mel_max = hz_to_mel(sample_rate / 2)  # Nyquist frequency
    mel_points = np.linspace(mel_min, mel_max, n_mels + 2)  # Equally spaced in Mel scale
    hz_points = mel_to_hz(mel_points)  # Convert Mel points to Hz
    bin_points = np.floor((n_fft + 1) * hz_points / sample_rate).astype(int)  # FFT bin numbers
    
    filter_bank = np.zeros((n_mels, n_fft // 2 + 1))
    
    for i in range(1, n_mels + 1):
        filter_bank[i - 1, bin_points[i - 1]:bin_points[i]] = \
            (np.arange(bin_points[i - 1], bin_points[i]) - bin_points[i - 1]) / (bin_points[i] - bin_points[i - 1])
        filter_bank[i - 1, bin_points[i]:bin_points[i + 1]] = \
            (bin_points[i + 1] - np.arange(bin_points[i], bin_points[i + 1])) / (bin_points[i + 1] - bin_points[i])
    
    return filter_bank

def mel_spectrogram(signal, sample_rate, n_mels=128, n_fft=2048, hop_length=512):
    # Adjust the noverlap to be less than nperseg
    noverlap = n_fft - hop_length if hop_length < n_fft else n_fft // 2
    
    # Compute the Short-Time Fourier Transform (STFT)
    frequencies, times, spectrogram = scipy.signal.stft(signal, fs=sample_rate, window='hann',
                                                        nperseg=n_fft, noverlap=noverlap, nfft=n_fft)
    
    # Compute power spectrogram (magnitude squared)
    power_spectrogram = np.abs(spectrogram)**2
    
    # Create the Mel filter bank and apply it
    mel_filter = mel_filter_bank(sample_rate, n_fft, n_mels)
    mel_spectrogram = np.dot(mel_filter, power_spectrogram[:n_fft // 2 + 1])
    
    # Convert power to decibels (log scale)
    mel_spectrogram_db = 10 * np.log10(mel_spectrogram + 1e-9)  # Add small value to avoid log(0)
    
    return mel_spectrogram_db, times


def plot_mel_spectrogram(mel_spectrogram, times, sample_rate):
    plt.figure(figsize=(10, 6))
    plt.imshow(mel_spectrogram, aspect='auto', origin='lower', cmap='coolwarm',
               extent=[times.min(), times.max(), 0, sample_rate // 2])
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plt.show()

for instrument in all_instrument_classes:
    for wav_file, (sample_rate, signal) in instrument.signals_raw.items():
        instrument.signals_envelope[wav_file] = mel_spectrogram(signal, sample_rate)

ValueError: noverlap must be less than nperseg.

## Neural network

Lorem ipsum dolor sit amet.

In [ ]:
# x_train = signal
# y_train = instrument

model = Sequential()
model.add(Dense(units=32, activation='Leaky ReLU'))
